<a href="https://colab.research.google.com/github/hanzila1/POC--GenEE-Generative-Earth-Engine-/blob/main/POC_GenEE_(Generative_Earth_Engine).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

import threading
import subprocess
import time

def run_ollama_serve():
    subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

!pip install geopy
!pip install -qU langchain_ollama
!pip install leafmap

# Initialize Earth Engine
import ee
ee.Authenticate()
ee.Initialize(project='ee-hanzilabinyounus')

!ollama pull gemma2:2b


In [ ]:

import threading
import subprocess
import time
import ee
from langchain_ollama import OllamaLLM
import ipyleaflet
from ipyleaflet import Map, Marker, basemaps, TileLayer, DrawControl, MeasureControl, FullScreenControl, LayersControl
from ipywidgets import Layout, HTML, Text, Box, VBox
from IPython.display import display, clear_output
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

# Make sure Earth Engine is initialized
print("Verifying Earth Engine initialization...")
try:
    # This will throw an error if Earth Engine is not initialized
    ee.Image(0)
    print("Earth Engine is already initialized")
except:
    print("Initializing Earth Engine...")
    ee.Initialize(project='ee-hanzilabinyounas')
    print("Earth Engine initialized with project: ee-my-learning47")

# Earth Engine visualization functions
def add_sentinel_rgb(map_obj, lat, lon):
    """Add Sentinel-2 RGB imagery to an ipyleaflet map"""
    point = ee.Geometry.Point([lon, lat])
    point_geometry = point.buffer(100000)
    start_date = '2024-09-01'
    end_date = '2024-09-30'

    s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                    .filterBounds(point_geometry)
                    .filterDate(start_date, end_date)
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))

    median_composite = s2_collection.median()
    rgb_image = median_composite.select(['B4', 'B3', 'B2'])

    vis_params = {
        'bands': ['B4', 'B3', 'B2'],
        'min': 0,
        'max': 3000,
        'gamma': 1.2
    }

    map_id = rgb_image.getMapId(vis_params)
    sentinel_layer = TileLayer(
        url=map_id['tile_fetcher'].url_format,
        attribution='Google Earth Engine | Sentinel-2',
        name='Sentinel-2 RGB'
    )
    map_obj.add_layer(sentinel_layer)
    return sentinel_layer

def add_surface_water(map_obj, lat, lon):
    """Add Global Surface Water visualization to an ipyleaflet map"""
    point = ee.Geometry.Point([lon, lat])
    point_geometry = point.buffer(200000)

    # Load the Global Surface Water dataset
    gsw = ee.Image('JRC/GSW1_4/GlobalSurfaceWater')

    # Get the water occurrence layer (frequency of water presence)
    water_occurrence = gsw.select('occurrence')

    vis_params = {
        'min': 0,
        'max': 100,
        'palette': [
            '#ffffff',  # white for no water
            '#d4e7ff',  # very light blue
            '#a8d1ff',  # light blue
            '#7cbaff',  # medium-light blue
            '#51a3ff',  # medium blue
            '#258cff',  # medium-dark blue
            '#0075ff',  # dark blue
            '#005ebf',  # very dark blue
            '#004080'   # darkest blue for permanent water
        ]
    }

    map_id = water_occurrence.getMapId(vis_params)
    water_layer = TileLayer(
        url=map_id['tile_fetcher'].url_format,
        attribution='Google Earth Engine | Global Surface Water',
        name='Surface Water'
    )
    map_obj.add_layer(water_layer)
    return water_layer

def add_sentinel_ndvi(map_obj, lat, lon):
    """Add Sentinel-2 NDVI visualization to an ipyleaflet map"""
    point = ee.Geometry.Point([lon, lat])
    point_geometry = point.buffer(100000)
    start_date = '2024-09-01'
    end_date = '2024-09-30'

    s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                    .filterBounds(point_geometry)
                    .filterDate(start_date, end_date)
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)))

    median_composite = s2_collection.median()

    # Calculate NDVI
    ndvi = median_composite.normalizedDifference(['B8', 'B4'])

    vis_params = {
        'min': -0.2,
        'max': 0.8,
        'palette': [
            '#FFFFFF',  # white for low NDVI
            '#F4F9F4',  # very light green
            '#E8F4E8',  # lighter green
            '#DCF0DC',  # light green
            '#C2E3C2',  # medium-light green
            '#A8D5A8',  # medium green
            '#8EC78E',  # medium-dark green
            '#74BA74',  # darker green
            '#5AAD5A',  # very dark green
            '#006400'   # darkest green for highest NDVI
        ]
    }

    map_id = ndvi.getMapId(vis_params)
    ndvi_layer = TileLayer(
        url=map_id['tile_fetcher'].url_format,
        attribution='Google Earth Engine | Sentinel-2 NDVI',
        name='Sentinel-2 NDVI'
    )
    map_obj.add_layer(ndvi_layer)
    return ndvi_layer

# Map GUI class
class MapGUI:
    def __init__(self):
        # Initialize the LLM
        self.llm = OllamaLLM(model="gemma2:2b")

        # Create the map centered on a default location with ESRI World Imagery basemap
        self.map = Map(
            center=(10, 0),  # Default coordinates
            zoom=3,
            layout=Layout(height='500px', width='100%'),
            basemap=basemaps.Esri.WorldImagery
        )

        # Add DrawControl for drawing shapes
        draw_control = DrawControl(
            marker={'shapeOptions': {'color': '#0000FF'}},
            rectangle={'shapeOptions': {'color': '#0000FF'}},
            circle={'shapeOptions': {'color': '#0000FF'}},
            polygon={'shapeOptions': {'color': '#0000FF'}},
            circlemarker={},
            polyline={'shapeOptions': {'color': '#0000FF'}},
        )
        self.map.add_control(draw_control)

        # Add MeasureControl for measuring distances and areas
        measure_control = MeasureControl(
            position='bottomleft',
            active_color='orange',
            primary_length_unit='kilometers'
        )
        self.map.add_control(measure_control)

        # Add FullScreenControl
        fullscreen_control = FullScreenControl()
        self.map.add_control(fullscreen_control)

        # Add LayersControl for managing different layers
        layers_control = LayersControl(position='topright')
        self.map.add_control(layers_control)

        # Add scale control
        self.map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))

        # Create the text input
        self.text_input = Text(
            placeholder='Enter your prompt here and press Enter',
            layout=Layout(width='100%')
        )

        # Create a header
        self.header = HTML(
            value="<h3>Interactive Map Interface</h3>",
            layout=Layout(margin='10px')
        )

        # Create a response display
        self.response_display = HTML(
            value="",
            layout=Layout(margin='10px')
        )

        # Add event handler for the text input
        self.text_input.on_submit(self.handle_submit)

        # Combine all elements in a vertical box
        self.container = VBox([
            self.header,
            self.text_input,
            self.response_display,
            self.map
        ])

    def handle_submit(self, sender):
        """Handle the submit event when user presses Enter"""
        prompt = self.text_input.value
        self.text_input.value = ''  # Clear the input

        try:
            # Updated analysis prompt to include water option
            analysis_prompt = f"""Analyze this prompt: '{prompt}'
            Determine two things:
            1. The location name (city and country)
            2. Whether it requests RGB imagery, NDVI visualization or surface water
            Format your response exactly like this example:
            Location: Vienna, Austria
            Processing: RGB
            Note: Processing should be either RGB, NDVI, or Surface WATER
            """

            response = self.llm.invoke(analysis_prompt)

            # Parse LLM response
            try:
                response_lines = response.strip().split('\n')
                location = response_lines[0].split(': ')[1].strip()
                processing_type = response_lines[1].split(': ')[1].strip().upper()
                print(processing_type)
                # Initialize geocoder
                geolocator = Nominatim(user_agent="my_map_application")

                # Get coordinates for the location
                try:
                    loc = geolocator.geocode(location)
                    if loc:
                        # Center map on the location
                        self.map.center = (loc.latitude, loc.longitude)
                        self.map.zoom = 11

                        # Add marker
                        marker = Marker(location=(loc.latitude, loc.longitude), draggable=False)
                        self.map.add_layer(marker)

                        # Add appropriate visualization based on processing type
                        if processing_type == 'RGB':
                            layer = add_sentinel_rgb(self.map, loc.latitude, loc.longitude)
                            processing_text = "RGB visualization"
                        elif processing_type == 'NDVI':
                            layer = add_sentinel_ndvi(self.map, loc.latitude, loc.longitude)
                            processing_text = "NDVI analysis"
                        elif processing_type == 'SURFACE WATER':
                            layer = add_surface_water(self.map, loc.latitude, loc.longitude)
                            processing_text = "Surface water mapping"
                        else:
                            # Default to RGB if processing type is not recognized
                            layer = add_sentinel_rgb(self.map, loc.latitude, loc.longitude)
                            processing_text = "RGB visualization (default)"

                        self.response_display.value = f"""
                        <p><strong>Prompt:</strong> {prompt}</p>
                        <p><strong>Location:</strong> {location}</p>
                        <p><strong>Processing:</strong> {processing_text}</p>
                        <p><strong>Coordinates:</strong> ({loc.latitude}, {loc.longitude})</p>
                        """
                    else:
                        self.response_display.value = f"<p>Could not find coordinates for {location}</p>"

                except GeocoderTimedOut:
                    self.response_display.value = f"<p>Geocoding service timed out for {location}</p>"

            except Exception as e:
                self.response_display.value = f"<p>Error parsing LLM response: {str(e)}</p>"
                print(f"LLM response: {response}")  # Print for debugging

        except Exception as e:
            self.response_display.value = f"<p>Error processing prompt: {str(e)}</p>"
            print(f"Exception: {str(e)}")  # Print for debugging

    def display(self):
        """Display the GUI"""
        display(self.container)

# Create and display the GUI
gui = MapGUI()
gui.display()